In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, ast
from google.colab import files

sns.set()

In [30]:
df_trees = pd.read_csv("tree.csv", encoding="ISO-8859-1")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
len(df_trees)

45709

In [32]:
df_trees.head(3)

,id,address_number,street,common_name,scientific_name,height,width,growth_space_length,growth_space_width,growth_space_type,diameter_base_height,stems,overhead_utilities,land_use,condition,stormwater_benefits_dollar_value,stormwater_benefits_runoff_elim,property_value_benefits_dollarvalue,property_value_benefits_leaf_surface_area,energy_benefits_electricity_dollar_value,energy_benefits_gas_dollar_value,air_quality_benfits_o3dep_dollar_value,air_quality_benfits_o3dep_lbs,air_quality_benfits_vocavd_dollar_value,air_quality_benfits_vocavd_lbs,air_quality_benfits_no2dep_dollar_value,air_quality_benfits_no2dep_lbs,air_quality_benfits_no2avd_dollar_value,air_quality_benfits_no2avd_lbs,air_quality_benfits_so2dep_dollar_value,air_quality_benfits_so2dep_lbs,air_quality_benfits_so2avd_dollar_value,air_quality_benfits_so2avd_lbs,air_quality_benfits_pm10depdollar_value,air_quality_benfits_pm10dep_lbs,air_quality_benfits_pm10avd_dollar_value,air_quality_benfits_pm10avd_lbs,air_quality_benfits_total_dollar_value,air_quality_benfits_total_lbs,co2_benefits_dollar_value,co2_benefits_sequestered_lbs,co2_benefits_sequestered_value,co2_benefits_avoided_lbs,co2_benefits_avoided_value,co2_benefits_decomp_lbs,co2_benefits_maint_lbs,co2_benefits_totalco2_lbs,overall_benefits_dollar_value,neighborhood,council_district,ward,tract,public_works_division,pli_division,police_zone,fire_zone,latitude,longitude
0,754166088,7428,MONTICELLO ST,Stump,Stump,0.0,0.0,10.0,2.0,Well or Pit,16.0,1.0,Yes,Vacant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homewood North,9.0,13.0,4.200313e+10,2.0,13.0,5.0,3-17,40.458169,-79.889724
1,1946899269,220,BALVER AVE,Linden: Littleleaf,Tilia cordata,0.0,0.0,99.0,99.0,Open or Unrestricted,22.0,0.0,No,Residential,NaN,13.946694,1743.336782,21.984813,36.538333,15.776457,61.068319,2.360847,0.514346,0.072138,0.031229,0.992384,0.216206,3.702306,0.806603,0.274901,0.078994,1.407724,0.404518,2.185328,0.262976,0.461810,0.055573,11.457438,2.370444,0.944601,115.328075,0.847431,277.540793,2.039370,-96.345471,-13.708796,282.814600,125.178322,Oakwood,2.0,28.0,4.200356e+10,5.0,28.0,6.0,1-19,40.429269,-80.067868
2,1431517397,2822,SIDNEY ST,Maple: Red,Acer rubrum,22.0,6.0,6.0,3.0,Well or Pit,6.0,1.0,No,Commercial/Industrial,Fair,3.974858,496.857276,51.529093,85.640354,3.388820,16.084741,0.464026,0.101095,0.017643,0.007638,0.200391,0.043658,0.875966,0.190842,0.058725,0.016875,0.302735,0.086993,0.444639,0.053507,0.110526,0.013300,2.474653,0.513908,0.314952,45.287950,0.332776,59.616419,0.438061,-6.868639,-3.738763,94.296967,77.767116,South Side Flats,3.0,16.0,4.200316e+10,3.0,16.0,3.0,4-24,40.426797,-79.965035


In [33]:
#Some datapoints are missing the basic tree name information. Only a few datapoints (13 in total)
#so decided to drop them. 
df_trees = df_trees.dropna(subset = ['common_name'])
df_trees = df_trees.dropna(subset = ['scientific_name'])

# There are around 300 datapoints that are missing its geo location info. Drop them. 
# Our team is interested in the neighborhood-level granularity so as long as the 
# data point contains neighborhood information, it is valuable to us. 
df_trees = df_trees.dropna(subset = ["latitude"])
df_trees = df_trees.dropna(subset = ["longitude"])
df_trees = df_trees.dropna(subset = ["neighborhood"])

print(len(df_trees))

45347


In [34]:
#These are all numerical benefit values. 
values = ['stormwater_benefits_dollar_value', 'stormwater_benefits_runoff_elim',
       'property_value_benefits_dollarvalue',
       'property_value_benefits_leaf_surface_area',
       'energy_benefits_electricity_dollar_value',
       'energy_benefits_gas_dollar_value',
       'air_quality_benfits_o3dep_dollar_value',
       'air_quality_benfits_o3dep_lbs',
       'air_quality_benfits_vocavd_dollar_value',
       'air_quality_benfits_vocavd_lbs',
       'air_quality_benfits_no2dep_dollar_value',
       'air_quality_benfits_no2dep_lbs',
       'air_quality_benfits_no2avd_dollar_value',
       'air_quality_benfits_no2avd_lbs',
       'air_quality_benfits_so2dep_dollar_value',
       'air_quality_benfits_so2dep_lbs',
       'air_quality_benfits_so2avd_dollar_value',
       'air_quality_benfits_so2avd_lbs',
       'air_quality_benfits_pm10depdollar_value',
       'air_quality_benfits_pm10dep_lbs',
       'air_quality_benfits_pm10avd_dollar_value',
       'air_quality_benfits_pm10avd_lbs',
       'air_quality_benfits_total_dollar_value',
       'air_quality_benfits_total_lbs', 'co2_benefits_dollar_value',
       'co2_benefits_sequestered_lbs', 'co2_benefits_sequestered_value',
       'co2_benefits_avoided_lbs', 'co2_benefits_avoided_value',
       'co2_benefits_decomp_lbs', 'co2_benefits_maint_lbs',
       'co2_benefits_totalco2_lbs', 'overall_benefits_dollar_value']

#Assume tree stumps has no benefit values so replace NaN with 0.0
cond = (df_trees["common_name"] == "Stump") | (df_trees["scientific_name"] == "Stump")

#Assume vacant sites has no benefit values so replace NaN with 0.0
cond2 = (df_trees["common_name"] == 'Vacant Site Small') | (df_trees["common_name"] == 'Vacant Site Medium') | (df_trees["common_name"] == 'Vacant Site Not Suitable') | (df_trees["common_name"] == 'Vacant Site Not Large') 

for val in values:
  df_trees.loc[cond,val] = df_trees.loc[cond,val].fillna(0.0)
  df_trees.loc[cond2,val] = df_trees.loc[cond2,val].fillna(0.0)

In [35]:
# Some trees are missing some of the height, width, or benefit values. 
# Replace them with the average for that tree type so we do not have to drop that tree datapoint.
# These values should be relatively independent of which neighborhood that tree is located in.
# For example, it is unlikely that there will be a statistically significant difference in height
# for the same type of tree across the neighborhoods. Similarly, the air quality value a type of tree provides
# should be independent of the neighborhood. 

independent_cols_to_replace = ['height', 'width', 'growth_space_length','growth_space_width','diameter_base_height','stems',     
                               'air_quality_benfits_o3dep_dollar_value',
       'energy_benefits_electricity_dollar_value',
       'energy_benefits_gas_dollar_value',
       'air_quality_benfits_o3dep_lbs',
       'air_quality_benfits_vocavd_dollar_value',
       'air_quality_benfits_vocavd_lbs',
       'air_quality_benfits_no2dep_dollar_value',
       'air_quality_benfits_no2dep_lbs',
       'air_quality_benfits_no2avd_dollar_value',
       'air_quality_benfits_no2avd_lbs',
       'air_quality_benfits_so2dep_dollar_value',
       'air_quality_benfits_so2dep_lbs',
       'air_quality_benfits_so2avd_dollar_value',
       'air_quality_benfits_so2avd_lbs',
       'air_quality_benfits_pm10depdollar_value',
       'air_quality_benfits_pm10dep_lbs',
       'air_quality_benfits_pm10avd_dollar_value',
       'air_quality_benfits_pm10avd_lbs',
       'air_quality_benfits_total_dollar_value',
       'air_quality_benfits_total_lbs', 'co2_benefits_dollar_value',
       'co2_benefits_sequestered_lbs', 'co2_benefits_sequestered_value',
       'co2_benefits_avoided_lbs', 'co2_benefits_avoided_value',
       'co2_benefits_decomp_lbs', 'co2_benefits_maint_lbs',
       'co2_benefits_totalco2_lbs']


tree_names = df_trees["common_name"].unique()
for val in independent_cols_to_replace:
  for tree_name in tree_names:
    t = df_trees[df_trees["common_name"] == tree_name]
    mean = t[val].mean()
    df_trees[val].fillna(value=mean, inplace=True)


In [36]:
# Some attributes,on the other hand, may depend on the neighborhood. For example,
# the property value benefits should be heavily influenced by the property value
# in that neighorhood. Similarly, stormwater benefits can vary across neighborhood
# based on the sewage condition. 
dependent_cols_to_replace = ['stormwater_benefits_dollar_value', 'stormwater_benefits_runoff_elim','property_value_benefits_dollarvalue', 'property_value_benefits_leaf_surface_area', 'overall_benefits_dollar_value']

#Build a dictionary that maps neighborhood, tree name, and the means of the different benefit values.
neighborhoods = df_trees["neighborhood"].unique()
tree_name_neighbor_hood_value_average = {}

for neighborhood in neighborhoods:
  tree_name_neighbor_hood_value_average[neighborhood] = {}
  t = df_trees[df_trees["neighborhood"] == neighborhood]
  for tree_name in tree_names:
    tree_name_neighbor_hood_value_average[neighborhood][tree_name] = {}
    k = t[t["common_name"] == tree_name]
    for val in dependent_cols_to_replace:
      mean = k[val].mean()
      tree_name_neighbor_hood_value_average[neighborhood][tree_name][val] = mean

In [37]:
# This cell may take a while to run since it is a triple for-loop. (around 10 mins in Google CoLab)
for neighborhood in neighborhoods:
  for tree_name in tree_names:
    for val in dependent_cols_to_replace:
      mean = tree_name_neighbor_hood_value_average[neighborhood][tree_name][val]
      mask = (df_trees['neighborhood'] == neighborhood) & (df_trees['common_name'] == tree_name)
      df_trees.loc[mask, val] = mean

In [38]:
print(df_trees.columns[df_trees.isnull().any()])

Index(['address_number', 'street', 'growth_space_type', 'overhead_utilities',
       'land_use', 'condition', 'stormwater_benefits_dollar_value',
       'stormwater_benefits_runoff_elim',
       'property_value_benefits_dollarvalue',
       'property_value_benefits_leaf_surface_area',
       'overall_benefits_dollar_value', 'council_district', 'ward',
       'pli_division'],
      dtype='object')


In [39]:
# However, some of the data points may still contain NaN in those columns 
# because that data point is the only tree of that type in that neighborhood. 
# In that case, just use the entire average for that type of tree. 
for val in dependent_cols_to_replace:
  for tree_name in tree_names:
    t = df_trees[df_trees["common_name"] == tree_name]
    mean = t[val].mean()
    df_trees[val].fillna(value=mean, inplace=True)


print(df_trees.columns[df_trees.isnull().any()])
# There will be some data points that contain NaN in some columns in the end, 
# but those can be processed specifically when those columns are used. 
# Also, data points like "Vacant Site" make sense to contain NaN because there is
# no tree there yet. 

#There are around 4700 entries where the "tree" is actually just a tree stump or 
#a vacant spot of various sizes. However these data are still interesting. 
temp = df_trees[(df_trees["common_name"] == "Stump") | (df_trees["common_name"] == 'Vacant Site Small') | (df_trees["common_name"] == 'Vacant Site Medium') | (df_trees["common_name"] == 'Vacant Site Not Suitable') | (df_trees["common_name"] == 'Vacant Site Not Large')] 
print(len(temp))

Index(['address_number', 'street', 'growth_space_type', 'overhead_utilities',
       'land_use', 'condition', 'council_district', 'ward', 'pli_division'],
      dtype='object')
4708


In [40]:
print(len(df_trees))
# Download the cleaned data 
df_trees.to_csv('cleaned_tree_data_3.csv', index=False) 
files.download('cleaned_tree_data_3.csv')

45347


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>